In [2]:
!pip install transformers torch

In [2]:
!unzip Project-Pattern-Recognition-main.zip

Archive:  Project-Pattern-Recognition-main.zip
replace __MACOSX/._Project-Pattern-Recognition-main? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [1]:
import os
import glob

def get_all_py_files(directory):
    # Using glob to recursively find all Python files
    py_files = glob.glob(os.path.join(directory, '**', '*.py'), recursive=True)
    py_files = [file for file in py_files if not file.endswith("__init__.py")]
    return py_files

# Example usage
directory = './Project-Pattern-Recognition-main'  # Replace this with the path of your directory
py_files = get_all_py_files(directory)

# Print out the list of .py files found
for file in py_files:
    print(file)


./Project-Pattern-Recognition-main/data_insertion.py
./Project-Pattern-Recognition-main/imports.py
./Project-Pattern-Recognition-main/Final.py
./Project-Pattern-Recognition-main/feature_selection/feature_selection.py
./Project-Pattern-Recognition-main/illumination_preprocessing/illumination_preprocessing.py
./Project-Pattern-Recognition-main/dataloader/dataloader.py
./Project-Pattern-Recognition-main/feature_extraction/feature_extraction.py
./Project-Pattern-Recognition-main/preprocessing/clustering_segmentation.py
./Project-Pattern-Recognition-main/preprocessing/region_segmentation.py
./Project-Pattern-Recognition-main/preprocessing/preproccessing.py
./Project-Pattern-Recognition-main/preprocessing/edge_detection.py
./Project-Pattern-Recognition-main/preprocessing/image_aligner.py
./Project-Pattern-Recognition-main/preprocessing/threshold_segmentation.py
./Project-Pattern-Recognition-main/preprocessing/image_restoration.py
./Project-Pattern-Recognition-main/model_selection/model_selec

In [3]:
import random

def read_file(file_path):
    with open(file_path, 'r') as f:
        return f.readlines()


def split_code_by_lines_only(code_lines, num_splits=30):
    examples = []
    for _ in range(num_splits):
        # Pick a random line to simulate the cursor position (prefix)
        line_idx = random.randint(1, len(code_lines) - 2)

        # Split code for prefix: All code before the cursor line
        prefix = ''.join(code_lines[:line_idx])

        # Ensure the suffix line is after line_idx
        if line_idx < len(code_lines) - 1:
            suffix_line_idx = random.randint(line_idx + 1, len(code_lines) - 1)
        else:
            suffix_line_idx = len(code_lines) - 1  # Fallback if near the end

        # Suffix: All code after the suffix line
        suffix = ''.join(code_lines[suffix_line_idx:])

        # Middle: All lines between the prefix and the suffix
        middle = ''.join(code_lines[line_idx:suffix_line_idx])

        examples.append((prefix, middle, suffix))

    return examples

def split_code(code_lines, num_splits=30):
    examples = []
    for _ in range(num_splits):
        # Pick a random line to simulate the cursor position
        line_idx = random.randint(1, len(code_lines) - 2)
        line = code_lines[line_idx]

        # Randomly select a point within that line to split
        if len(line) > 1:
            cursor_pos = random.randint(0, len(line) - 1)
        else:
            cursor_pos = 0

        # Split code at the cursor position (for prefix)
        prefix = ''.join(code_lines[:line_idx]) + line[:cursor_pos]

        # Ensure the suffix line is after line_idx
        if line_idx < len(code_lines) - 1:
            suffix_line_idx = random.randint(line_idx + 1, len(code_lines) - 1)
        else:
            suffix_line_idx = len(code_lines) - 1  # Fallback in case the cursor is near the end

        suffix_line = code_lines[suffix_line_idx]

        # Randomly select a point within that line to split
        if len(suffix_line) > 1:
            suffix_cursor_pos = random.randint(0, len(suffix_line) - 1)
        else:
            suffix_cursor_pos = 0

        # The suffix starts from the selected line and continues onward
        suffix = suffix_line[suffix_cursor_pos:] + ''.join(code_lines[suffix_line_idx + 1:])

        # The middle is the code between the cursor and the start of the suffix
        middle = line[cursor_pos:] + ''.join(code_lines[line_idx + 1:suffix_line_idx]) + suffix_line[:suffix_cursor_pos]

        examples.append((prefix, middle, suffix))

    return examples

def create_dataset(py_files, num_examples_per_file=1):
    examples = []
    for file in py_files:
        code_lines = read_file(file)
        # examples.extend(split_code(code_lines, num_splits=num_examples_per_file))
        examples.extend(split_code_by_lines_only(code_lines, num_splits=num_examples_per_file))

    return examples

dataset = create_dataset(py_files, num_examples_per_file=3)

In [4]:
# Dataset is a list of tuples [(prefix, middle, suffix), ...]
for i, example in enumerate(dataset):
    print(f"Example {i+1}:")
    print("Prefix: \n", example[0])
    print("---------------------------------------------------------------")
    print("Middle (missing): \n", example[1])
    print("---------------------------------------------------------------")
    print("Suffix: \n", example[2])
    print("---------------------------------------------------------------")
    print("\n")

Example 1:
Prefix: 
 import os
import gdown

url = 'https://drive.google.com/uc?id=1JLxhdIddq6_vKlHml7jT48VaeXoJjvpR'
output_filename = 'data.zip'

# Get current working directory
cwd = os.getcwd()

# Concatenate current working directory and output filename
output_path = os.path.join(cwd, output_filename)

# Download file from Google Drive to output path
gdown.download(url, output_path, quiet=False)

import zipfile

with zipfile.ZipFile(output_filename, 'r') as zip_ref:

---------------------------------------------------------------
Middle (missing): 
     zip_ref.extractall('./data')

---------------------------------------------------------------
Suffix: 
 

---------------------------------------------------------------


Example 2:
Prefix: 
 import os
import gdown

url = 'https://drive.google.com/uc?id=1JLxhdIddq6_vKlHml7jT48VaeXoJjvpR'

---------------------------------------------------------------
Middle (missing): 
 output_filename = 'data.zip'

# Get current working director

In [7]:
from huggingface_hub import login
import json
from transformers import AutoModelForSequenceClassification, AutoModelForTokenClassification, AutoTokenizer
import torch
import numpy as np

In [8]:
#login huggingface

# Log in using the token from the environment variable
login(token=os.getenv("HF_TOKEN"))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [9]:
# Sane hyper-parameters
params = {
    'max_new_tokens': 200,
    'temperature': 0.2,
    'top_k': 50,
    'top_p': 0.1,
    'repetition_penalty': 1.17
}
device = "cuda"


In [10]:
# Tested the model using tiny_starcoder before but found out CodeLlama is much better, thats why its commented

# from transformers import AutoModelForCausalLM, AutoTokenizer
# checkpoint = "bigcode/tiny_starcoder_py"
# tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# model = AutoModelForCausalLM.from_pretrained(checkpoint).to(device)

In [11]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer
)

base_model_id = "codellama/CodeLlama-7b-hf"

tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=None,
    device_map=None,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [67]:
# Trying a demo on filling first
prompt = '''def hello_world():
    <FILL_ME>
'''

input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"].to("cuda")
outputs = model.generate(
    input_ids,
    pad_token_id=tokenizer.eos_token_id,
    **params
)

filling = tokenizer.decode(outputs[0], skip_special_tokens=False)
print(filling)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


<s> <PRE> def hello_world():
     <SUF>
 <MID>print("Hello World!") <EOT></s>


In [14]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

def extract_middle(output_text):
    # Ensure that both the <fim_middle> and <|endoftext|> tokens are in the output
    if "<MID>" in output_text:
        # Find the starting point after <fim_middle> and ending before <|endoftext|>
        start = output_text.index("<MID>") + len("<MID>")
        if "<EOT>" in output_text:
          end = output_text.index("<EOT>")
        else:
          end = len(output_text)
        generated_middle = output_text[start:end].strip()  # Extract and strip any surrounding whitespace
        return generated_middle
    else:
        return None

# Function to apply fill-in-the-middle using StarCoder
def apply_fill_in_the_middle(dataset):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)

    results = []

    for i, (prefix, middle, suffix) in enumerate(dataset):
        # Prepare FIM input for the model
        fim_input = f"{prefix}<FILL_ME>{suffix}"
        # Tokenize the input
        inputs = tokenizer.encode(fim_input, return_tensors="pt").to(device)

        # Generate the middle part
        outputs = model.generate(inputs, pad_token_id=tokenizer.eos_token_id, **params)

        # Decode the output to get the generated middle code
        generated_middle = tokenizer.decode(outputs[0], skip_special_tokens=False)

        # Save the result (original middle, generated middle)
        results.append({
            "prefix": prefix,
            "suffix": suffix,
            "actual_middle": middle,
            "generated_middle": extract_middle(generated_middle),
        })

    return results

# Applying fill-in-the-middle on the dataset
results = apply_fill_in_the_middle(dataset[5:10])

# Print the results
for i, result in enumerate(results):
    print(f"Example {i+1}:")
    print("Prefix: \n", result['prefix'])
    print("Actual Middle: \n", result['actual_middle'])
    print("Generated Middle: \n", result['generated_middle'])
    print("Suffix: \n", result['suffix'])
    print("\n")


Example 1:
Prefix: from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from keras import regularizers

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.cluster import KMeans

from skimage.filters import sobel, prewitt, roberts, laplace , median, gaussian, threshold_otsu, rank, threshold_local
from skimage.feature import canny
from skimage.restoration import denoise_nl_means, wiener
from skimage.morphology import disk , square
from skimage.draw import rectangle

Actual Middle 
 from skimage import exposure, filters
from skimage.segmentation import f

# Evaluation Section

In [64]:
from nltk.translate.chrf_score import sentence_chrf
from transformers import RobertaTokenizer, RobertaModel
import torch


def exact_match(actual_middle, generated_middle):
    return int(actual_middle.strip() == generated_middle.strip())

def chrf_score(actual_middle, generated_middle):
    return sentence_chrf(actual_middle, generated_middle)

from nltk.translate.bleu_score import sentence_bleu

def bleu_score(actual_middle, generated_middle):
    reference = [list(actual_middle)]
    candidate = list(generated_middle)
    return sentence_bleu(reference, candidate)

from difflib import SequenceMatcher

def levenshtein_ratio(actual_middle, generated_middle):
    return SequenceMatcher(None, actual_middle, generated_middle).ratio()

import ast

def is_syntax_valid(generated_middle):
    try:
        ast.parse(generated_middle)
        return 1  # Valid code
    except SyntaxError:
        return 0  # Invalid code

# Load CodeBERT
eval_tokenizer = RobertaTokenizer.from_pretrained('microsoft/codebert-base')
eval_model = RobertaModel.from_pretrained('microsoft/codebert-base')

device = "cuda" if torch.cuda.is_available() else "cpu"
eval_model.to(device)

def get_code_embedding(code):
    inputs = eval_tokenizer(code, return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        outputs = eval_model(**inputs)
        # Take the mean of the token embeddings for the final representation
        return outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()
from sklearn.metrics.pairwise import cosine_similarity

def code_similarity(actual_middle, generated_middle):
    actual_embedding = get_code_embedding(actual_middle)
    generated_embedding = get_code_embedding(generated_middle)
    similarity = cosine_similarity([actual_embedding], [generated_embedding])
    return similarity[0][0]  # Extract the similarity value


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [66]:
from statistics import mean

def evaluate_model(results):
    exact_match_scores = []
    chrf_scores = []
    bleu_scores = []
    levenshtein_scores = []
    syntactic_similarity_scores = []

    for result in results:
        actual_middle = result['actual_middle']
        generated_middle = result['generated_middle'] or ""

        exact_match_scores.append(exact_match(actual_middle, generated_middle))
        chrf_scores.append(chrf_score(actual_middle, generated_middle))
        bleu_scores.append(bleu_score(actual_middle, generated_middle))
        levenshtein_scores.append(levenshtein_ratio(actual_middle, generated_middle))
        syntactic_similarity_scores.append(code_similarity(actual_middle, generated_middle))

    return {
        "Exact Match": mean(exact_match_scores),
        "chrF Score": mean(chrf_scores),
        "BLEU Score": mean(bleu_scores),
        "Levenshtein Ratio": mean(levenshtein_scores),
        "Syntactic Similarity": mean(syntactic_similarity_scores)
    }

# Run evaluation on the results
evaluation = evaluate_model(results)
print(evaluation)

{'Exact Match': 0, 'chrF Score': 0.32759649710376354, 'BLEU Score': 0.32761429252470736, 'Levenshtein Ratio': 0.11234714383277125, 'Syntactic Similarity': 0.8496151}


# Explaining Evaluation
#1. Exact Match: 0

Interpretation: An exact match score of 0 indicates that none of the generated middle segments perfectly match the actual middle segments across your dataset. This could suggest that while the model is generating relevant content, it is not producing the exact code that was intended. This is often the case in creative tasks like code generation, where multiple valid implementations may exist.

#2. chrF Score: 0.3276

Interpretation: A chrF score of 0.3276 indicates a moderate level of similarity between the generated and actual code snippets at the character level. This score suggests that there are some overlapping character sequences, indicating that the generated code is somewhat aligned with the intended output, but there is still room for improvement.

#3. BLEU Score: 0.3276

Interpretation: A BLEU score of 0.3276 suggests that there is a moderate level of overlap between the n-grams of the generated and actual code segments. Similar to the chrF score, this indicates that while the generated code has some valid segments, it does not fully capture the intended middle segments.

#4. Levenshtein Ratio: 0.1123

Interpretation: A Levenshtein ratio of 0.1123 indicates a low level of similarity between the actual and generated middle segments. This low score suggests that the generated outputs are relatively far from the actual outputs, requiring considerable edits to align with the intended code. This might point to substantial differences in structure or content.

#5. Syntactic Similarity: 0.8496

Interpretation: A syntactic similarity score of 0.8496 indicates a high degree of structural similarity between the generated and actual code. This suggests that while the exact wording may differ, the generated code retains a similar syntactic form to the expected output, which is a positive outcome in code generation tasks.


# Overall Insights:
The low exact match score indicates that the model struggles to generate exact matches but still produces content that is relevant or structurally similar.

The chrF and BLEU scores show some level of quality and coherence in the generated outputs, but they also indicate that improvements are needed.

The high syntactic similarity score suggests that the model can capture the underlying structure of the code well, which may be more valuable in programming tasks where there can be multiple valid implementations of a function.